ITINERARY CHOICE

In [2]:
import gzip, os, pandas as pd
import larch.numba as lx

c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\sharrow\accessors.py:25: AccessorRegistrationWarning: registration of accessor <function register_dataarray_method.<locals>.wrapper at 0x0000021EC5FDA430> under name 'to_zarr' for type <class 'xarray.core.dataarray.DataArray'> is overriding a preexisting attribute with the same name.
  return xr.register_dataarray_accessor(func.__name__)(wrapper)
c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\numba\model.py:23: UserWarning: 

### larch.numba is experimental, and not feature-complete ###
 the first time you import on a new system, this package will
 compile optimized binaries for your machine, which may take 
 a little while, please be patient 

  warnings.warn( ### EXPERIMENTAL ### )


- In this example we will import the air itinerary choice example dataset, starting from a csv text file in idca format. 

- Suppose that data file is gzipped, named “arc.csv.gz” and is located in the current directory (use os.getcwd to see what is the current directory).

In [3]:
with gzip.open(lx.example_file("arc"), 'rt') as previewfile:
    print(*(next(previewfile) for x in range(70)))

id_case,id_alt,choice,traveler,origin,destination,nb_cnxs,elapsed_time,fare_hy,fare_ly,equipment,carrier,timeperiod
 1,1,0,1,444,222,1,300,470.55658,0,1,3,7
 1,2,0,1,444,222,1,345,475.92258,0,2,3,5
 1,3,0,1,444,222,1,335,443.48166,0,1,3,2
 1,4,0,1,444,222,1,435,433.56735,0,2,3,2
 1,5,0,1,444,222,1,710,449.83664,0,2,3,2
 1,6,0,1,444,222,1,380,470.45175,0,1,3,5
 1,7,0,1,444,222,1,345,440.70526,0,2,3,6
 1,8,0,1,444,222,1,320,474.57831,0,2,3,4
 1,9,0,1,444,222,1,335,474.97363,0,2,3,3
 1,10,0,1,444,222,1,335,481.98392,0,1,3,3
 1,11,0,1,444,222,1,320,440.41138,0,1,3,7
 1,12,0,1,444,222,1,360,455.11444,0,2,3,1
 1,13,0,1,444,222,1,380,470.67239,0,1,3,4
 1,14,14,1,444,222,0,215,317.4277,0,2,3,1
 1,15,19,1,444,222,0,215,283.96292,0,2,3,1
 1,16,19,1,444,222,0,215,285.04138,0,2,3,2
 1,17,19,1,444,222,0,215,283.59644,0,2,3,2
 1,18,1,1,444,222,0,220,276.40555,0,2,3,3
 1,19,8,1,444,222,0,220,285.51282,0,2,3,3
 1,20,10,1,444,222,0,215,313.89828,0,2,3,3
 1,21,7,1,444,222,0,220,280.06757,0,2,3,4
 1,22,1

About the Data
- The first line of the file contains column headers. After that, each line represents an alternative available to one or more decision makers.

-  In our sample data, we see the first 67 lines of data share a id_case of 1, indicating that they are 67 different itineraries available to the first decision maker type. 

- An identidier of the alternatives is given by the number in the column id_alt, although this number is simply a sequential counter within each case in the raw data, and conveys no other information about the itinerary or its attributes. 

- The observed choices of the decision maker[s] are indicated in the column choice. That column counts the number of travelers who face this choice set and chose the itinerary described by this row in the file.


Dimensions:


- id_case: 105 unique cases (e.g., travelers, individuals).
- id_alt: 127 unique alternatives for each case.


Variables:

- choice: The chosen alternative for each case.

- traveler: Identifier for the traveler associated with each case.

- origin: Origin of the travel for each case.

- destination: Destination of the travel for each case.

- nb_cnxs: Number of connections for each alternative in each case.

- elapsed_time: Elapsed time for each alternative in each case.

- fare_hy: High fare for each alternative in each case.

- fare_ly: Low fare for each alternative in each case.

- equipment: Equipment type for each alternative in each case.

- carrier: Carrier for each alternative in each case.

- timeperiod: Time period for each alternative in each case.

- _avail_: Availability flag for each alternative in each case.


In [42]:
itin = pd.read_csv(lx.example_file("arc"), index_col=['id_case','id_alt'])
itin.info() 
#We can load this data easily using pandas. We’ll also set the index of the 
#resulting DataFrame to be the case and alt identifiers.

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6023 entries, (1, 1) to (105, 51)
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   choice        6023 non-null   int64  
 1   traveler      6023 non-null   int64  
 2   origin        6023 non-null   int64  
 3   destination   6023 non-null   int64  
 4   nb_cnxs       6023 non-null   int64  
 5   elapsed_time  6023 non-null   int64  
 6   fare_hy       6023 non-null   float64
 7   fare_ly       6023 non-null   float64
 8   equipment     6023 non-null   int64  
 9   carrier       6023 non-null   int64  
 10  timeperiod    6023 non-null   int64  
dtypes: float64(2), int64(9)
memory usage: 545.3 KB


In [43]:
d = lx.Dataset.construct.from_idca(itin, crack=True)
d
#created an xarray dataset
#attributes are the indices

c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\xarray\core\duck_array_ops.py:216: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\dataset\flow.py:339: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  return self.dims[self.CASEID]
c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\dataset\flow.py:475: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  return self._obj.dims[self.ALTID]


<xarray.Dataset> Size: 871kB
Dimensions:       (id_case: 105, id_alt: 127)
Coordinates:
  * id_case       (id_case) int64 840B 1 2 3 4 5 6 7 ... 100 101 102 103 104 105
  * id_alt        (id_alt) int64 1kB 1 2 3 4 5 6 7 ... 122 123 124 125 126 127
Data variables:
    choice        (id_case, id_alt) int64 107kB 0 0 ... -9223372036854775808
    traveler      (id_case) int64 840B 1 2 1 2 1 2 1 2 1 2 ... 2 1 2 2 1 2 2 1 2
    origin        (id_case) int64 840B 444 444 444 444 444 ... 777 777 777 777
    destination   (id_case) int64 840B 222 222 222 222 222 ... 111 111 111 111
    nb_cnxs       (id_case, id_alt) int64 107kB 1 1 ... -9223372036854775808
    elapsed_time  (id_case, id_alt) int64 107kB 300 345 ... -9223372036854775808
    fare_hy       (id_case, id_alt) float64 107kB 470.6 475.9 443.5 ... nan nan
    fare_ly       (id_case, id_alt) float64 107kB 0.0 0.0 0.0 ... nan nan nan
    equipment     (id_case, id_alt) int64 107kB 1 2 ... -9223372036854775808
    carrier       (id_case, id_alt) int64 107kB 3 3 ... -9223372036854775808
    timeperiod    (id_case, id_alt) int64 107kB 7 5 ... -9223372036854775808
    _avail_       (id_case, id_alt) int8 13kB 1 1 1 1 1 1 1 1 ... 0 0 0 0 0 0 0
Attributes:
    _caseid_:  id_case
    _altid_:   id_alt

 - By setting crack to True, Larch automatically analyzed the data to find variables that do not vary within cases, and transformed those into |idco| format variables. 
 
- If you would prefer that Larch not do this you can omit this argument or set it to False. 

- This is particularly important for larger datasets (the data sample included is only a tiny extract of the data that might be available for this kind of model), as breaking the data into seperate |idca| and |idco| parts is a relatively expensive operation, and it is not actually required for most model structures.

In [7]:
type(d)

xarray.core.dataset.Dataset

- Also, you may note that in creating the Dataset object, the set of all possible alternatives was deduced automatically from all the values in the altid column. 

- You will note that, while the sample case we have peeked at in the beginning of the raw data file has 67 alternatives, there are other observations in the file with alternatives numbering up to 127.

- The variability in the alternatives is handled by using idce data format which is sparse idca case-alternative forma in which each record can include information on the variables related to the decision maker or the choice itself, the attributes of that particular alternative, and a choice variable that indicates whether the alternative was or was not chosen.
In the sparse idca the missing alternatives are just dropped.

MODEL USING MNL

In [44]:
d = lx.Dataset.construct.from_idce(
    pd.read_csv(lx.example_file("arc"), index_col=['id_case','id_alt']),
)

In [7]:
d

<xarray.Dataset> Size: 403kB
Dimensions:       (_casealt_: 6023, id_case: 105, id_alt: 127, _caseptr_: 106)
Coordinates:
  * id_case       (id_case) int64 840B 1 2 3 4 5 6 7 ... 100 101 102 103 104 105
  * id_alt        (id_alt) int64 1kB 1 2 3 4 5 6 7 ... 122 123 124 125 126 127
    alt_idx       (_casealt_) int16 12kB 0 1 2 3 4 5 6 ... 44 45 46 47 48 49 50
  * _caseptr_     (_caseptr_) int64 848B 0 67 134 200 ... 5885 5921 5972 6023
Dimensions without coordinates: _casealt_
Data variables:
    choice        (_casealt_) int64 48kB 0 0 0 0 0 0 0 ... 74 61 30 73 59 32 52
    traveler      (id_case) int64 840B 1 2 1 2 1 2 1 2 1 2 ... 2 1 2 2 1 2 2 1 2
    origin        (id_case) int64 840B 444 444 444 444 444 ... 777 777 777 777
    destination   (id_case) int64 840B 222 222 222 222 222 ... 111 111 111 111
    nb_cnxs       (_casealt_) int64 48kB 1 1 1 1 1 1 1 1 1 ... 0 0 0 0 0 0 0 0 0
    elapsed_time  (_casealt_) int64 48kB 300 345 335 435 710 ... 120 120 120 120
    fare_hy       (_casealt_) float64 48kB 470.6 475.9 443.5 ... 0.0 0.0 0.0
    fare_ly       (_casealt_) float64 48kB 0.0 0.0 0.0 0.0 ... 129.3 152.3 159.9
    equipment     (_casealt_) int64 48kB 1 2 1 2 2 1 2 2 2 ... 2 2 2 2 2 2 2 2 2
    carrier       (_casealt_) int64 48kB 3 3 3 3 3 3 3 3 3 ... 5 5 5 5 5 5 5 5 5
    timeperiod    (_casealt_) int64 48kB 7 5 2 2 2 5 6 4 3 ... 4 5 6 6 7 8 9 2 7
Attributes:
    _exclude_dims_:  ('id_case', 'id_alt', '_caseptr_')
    _caseid_:        id_case
    _altid_:         id_alt
    _casealt_:       _casealt_
    _alt_idx_:       alt_idx
    _caseptr_:       _caseptr_

In [9]:
#Now let’s make our model. We’ll use a few variables 
#to define our linear-in-parameters utility function.

m = m = lx.Model(datatree=d)

# Define some variable
v = [
    "timeperiod==2",
    "timeperiod==3",
    "timeperiod==4",
    "timeperiod==5",
    "timeperiod==6",
    "timeperiod==7",
    "timeperiod==8",
    "timeperiod==9",
    "carrier==2",
    "carrier==3",
    "carrier==4",
    "carrier==5",
    "equipment==2",
    "fare_hy",    
    "fare_ly",    
    "elapsed_time",  
    "nb_cnxs",       
]

c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\dataset\flow.py:264: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  case_index = case_ptr_to_indexes(self._obj.dims[self.CASEALT], self[self.CASEPTR].values)


- The larch.roles module defines a few convenient classes for declaring data and parameter. 

- One we will use here is PX which creates a linear-in-parameter term that represents one data element (a column from our data, or an expression that can be evaluated on the data alone) multiplied by a parameter with the same name.

In [10]:
from larch.roles import PX

#The PX function is used to create parameter-variable pairs for each variable in v. 
#This means for each variable, PX(variable) creates a term in the utility function.
m.utility_ca = sum(PX(i) for i in v)

m.choice_ca_var = 'choice'
m.availability_var = 1

In [11]:
#To estimate the likelihood maximizing parameters
m.maximize_loglike()

c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\numba\data_arrays.py:475: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  model_dataset.dims.get(casealt_dim),
c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\dataset\__init__.py:1149: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  return self.root_dataset.dims[self.ALTID]
c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\dataset\__init__.py:1149: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArr

,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
carrier==2,0.117200,0.0,0.0,-inf,inf,0,,0.117200
carrier==3,0.638554,0.0,0.0,-inf,inf,0,,0.638554
carrier==4,0.565252,0.0,0.0,-inf,inf,0,,0.565252
carrier==5,-0.624022,0.0,0.0,-inf,inf,0,,-0.624022
elapsed_time,-0.006087,0.0,0.0,-inf,inf,0,,-0.006087
equipment==2,0.466305,0.0,0.0,-inf,inf,0,,0.466305
fare_hy,-0.001175,0.0,0.0,-inf,inf,0,,-0.001175
fare_ly,-0.001177,0.0,0.0,-inf,inf,0,,-0.001177
nb_cnxs,-2.947153,0.0,0.0,-inf,inf,0,,-2.947153
timeperiod==2,0.095949,0.0,0.0,-inf,inf,0,,0.095949


c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\dataset\flow.py:339: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  return self.dims[self.CASEID]
c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\numba\model.py:1789: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = 

┣          loglike: -777770.0688722525
┣                x: carrier==2       0.117200
┃                   carrier==3       0.638554
┃                   carrier==4       0.565252
┃                   carrier==5      -0.624022
┃                   elapsed_time    -0.006087
┃                   equipment==2     0.466305
┃                   fare_hy         -0.001175
┃                   fare_ly         -0.001177
┃                   nb_cnxs         -2.947153
┃                   timeperiod==2    0.095949
┃                   timeperiod==3    0.126533
┃                   timeperiod==4    0.060552
┃                   timeperiod==5    0.140963
┃                   timeperiod==6    0.238254
┃                   timeperiod==7    0.351391
┃                   timeperiod==8    0.353302
┃                   timeperiod==9   -0.010309
┃                   dtype: float64
┣        tolerance: 1.3256993596404166e-06
┣            steps: array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
┣          message: 'Optimization terminated successfully.'
┣     elapsed_time: datetime.timedelta(microseconds=209508)
┣           method: 'bhhh'
┣          n_cases: 105
┣ iteration_number: 11
┣          logloss: 7407.333989259548

In [12]:
m.calculate_parameter_covariance()

In [13]:
m.graph

In [14]:
m.parameter_summary()

C:\Users\jiyav\AppData\Local\Temp\ipykernel_12844\1075600533.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  m.parameter_summary()
C:\Users\jiyav\AppData\Local\Temp\ipykernel_12844\1075600533.py:1: FutureWarning: ChainedAssignmentError: 

,Value,Std Err,t Stat,Signif,Null Value
carrier==2,0.117,0.00869,13.49,***,0.00
carrier==3,0.639,0.00813,78.55,***,0.00
carrier==4,0.565,0.0176,32.18,***,0.00
carrier==5,-0.624,0.0130,-48.17,***,0.00
elapsed_time,-0.00609,0.000111,-54.63,***,0.00
equipment==2,0.466,0.00931,50.10,***,0.00
fare_hy,-0.00118,2.83e-05,-41.54,***,0.00
fare_ly,-0.00118,8.51e-05,-13.83,***,0.00
nb_cnxs,-2.95,0.0254,-115.82,***,0.00
timeperiod==2,0.0959,0.00948,10.12,***,0.00


In [15]:
d

<xarray.Dataset> Size: 403kB
Dimensions:       (_casealt_: 6023, id_case: 105, id_alt: 127, _caseptr_: 106)
Coordinates:
  * id_case       (id_case) int64 840B 1 2 3 4 5 6 7 ... 100 101 102 103 104 105
  * id_alt        (id_alt) int64 1kB 1 2 3 4 5 6 7 ... 122 123 124 125 126 127
    alt_idx       (_casealt_) int16 12kB 0 1 2 3 4 5 6 ... 44 45 46 47 48 49 50
  * _caseptr_     (_caseptr_) int64 848B 0 67 134 200 ... 5885 5921 5972 6023
Dimensions without coordinates: _casealt_
Data variables:
    choice        (_casealt_) int64 48kB 0 0 0 0 0 0 0 ... 74 61 30 73 59 32 52
    traveler      (id_case) int64 840B 1 2 1 2 1 2 1 2 1 2 ... 2 1 2 2 1 2 2 1 2
    origin        (id_case) int64 840B 444 444 444 444 444 ... 777 777 777 777
    destination   (id_case) int64 840B 222 222 222 222 222 ... 111 111 111 111
    nb_cnxs       (_casealt_) int64 48kB 1 1 1 1 1 1 1 1 1 ... 0 0 0 0 0 0 0 0 0
    elapsed_time  (_casealt_) int64 48kB 300 345 335 435 710 ... 120 120 120 120
    fare_hy       (_casealt_) float64 48kB 470.6 475.9 443.5 ... 0.0 0.0 0.0
    fare_ly       (_casealt_) float64 48kB 0.0 0.0 0.0 0.0 ... 129.3 152.3 159.9
    equipment     (_casealt_) int64 48kB 1 2 1 2 2 1 2 2 2 ... 2 2 2 2 2 2 2 2 2
    carrier       (_casealt_) int64 48kB 3 3 3 3 3 3 3 3 3 ... 5 5 5 5 5 5 5 5 5
    timeperiod    (_casealt_) int64 48kB 7 5 2 2 2 5 6 4 3 ... 4 5 6 6 7 8 9 2 7
Attributes:
    _exclude_dims_:  ('id_case', 'id_alt', '_caseptr_')
    _caseid_:        id_case
    _altid_:         id_alt
    _casealt_:       _casealt_
    _alt_idx_:       alt_idx
    _caseptr_:       _caseptr_

In [282]:
d.choice

<xarray.DataArray 'choice' (_casealt_: 6023)> Size: 48kB
array([ 0,  0,  0, ..., 59, 32, 52], dtype=int64)
Coordinates:
    alt_idx  (_casealt_) int16 12kB 0 1 2 3 4 5 6 7 ... 43 44 45 46 47 48 49 50
Dimensions without coordinates: _casealt_

In [16]:
m.title = 'Itinerary Model'

In [17]:
report = lx.Reporter(title=m.title)

In [18]:
report << '# Parameter Summary' << m.parameter_summary()

C:\Users\jiyav\AppData\Local\Temp\ipykernel_12844\1699917087.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  report << '# Parameter Summary' << m.parameter_summary()
C:\Users\jiyav\AppData\Local\Temp\ipykernel_12844\1699917087.py:1: Futu

,Value,Std Err,t Stat,Signif,Null Value
carrier==2,0.117,0.00869,13.49,***,0.00
carrier==3,0.639,0.00813,78.55,***,0.00
carrier==4,0.565,0.0176,32.18,***,0.00
carrier==5,-0.624,0.0130,-48.17,***,0.00
elapsed_time,-0.00609,0.000111,-54.63,***,0.00
equipment==2,0.466,0.00931,50.10,***,0.00
fare_hy,-0.00118,2.83e-05,-41.54,***,0.00
fare_ly,-0.00118,8.51e-05,-13.83,***,0.00
nb_cnxs,-2.95,0.0254,-115.82,***,0.00
timeperiod==2,0.0959,0.00948,10.12,***,0.00


In [19]:
report << "# Estimation Statistics" << m.estimation_statistics()

c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\dataset\flow.py:339: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  return self.dims[self.CASEID]


<xmle.Elem 'div' with 5 children>

In [20]:
report << "# Utility Functions" << m.utility_functions()

+ P('timeperiod==2') * X('timeperiod==2') + P('timeperiod==3') * X('timeperiod==3') + P('timeperiod==4') * X('timeperiod==4') + P('timeperiod==5') * X('timeperiod==5') + P('timeperiod==6') * X('timeperiod==6') + P('timeperiod==7') * X('timeperiod==7') + P('timeperiod==8') * X('timeperiod==8') + P('timeperiod==9') * X('timeperiod==9') + P('carrier==2') * X('carrier==2') + P('carrier==3') * X('carrier==3') + P('carrier==4') * X('carrier==4') + P('carrier==5') * X('carrier==5') + P('equipment==2') * X('equipment==2') + P.fare_hy * X.fare_hy + P.fare_ly * X.fare_ly + P.elapsed_time * X.elapsed_time + P.nb_cnxs * X.nb_cnxs


In [21]:
report.save(
    'Itinerary_choice.html',
    overwrite=True,
    metadata=m,
)

'Itinerary_choice.html'

SUMMARY

The aim of the code is to build and estimate a multinomial logit (MNL) model for itinerary choice using the Larch library. Here's a breakdown of the key steps:

- Loading Data: 

The code loads a dataset containing information about itinerary choices. This dataset includes variables such as time periods, carriers, equipment types, fares, elapsed times, and number of connections.

- Model Specification: 

It specifies the variables (v) to be used in the utility function of the MNL model. These variables include indicators for different time periods, carriers, equipment types, as well as continuous variables for fares, elapsed times, and number of connections.

- Model Building: 

It constructs an MNL model (m) using the specified dataset and utility function. The utility function is defined as the sum of linear-in-parameter terms for each variable in v.

- Estimation: 

It estimates the parameters of the MNL model by maximizing the log-likelihood function. This is done using the maximize_loglike() method of the model object (m).

- Results: 

After estimation, the code prints the estimated coefficients for each variable, along with other information such as log-likelihood value and optimization details.



Overall, the code aims to build an MNL model to understand and predict itinerary choices based on the provided dataset. The model estimates the influence of different factors (variables) on the probability of choosing a particular itinerary.

PREDICTING VALUES AND FINDING ACCURACY OF THE MODEL


In [2]:
import pandas as pd
import larch.numba as lx

itin = pd.read_csv(lx.example_file("arc"),index_col=['id_case','id_alt'])
#itin.set_index(['id_case','id_alt'],inplace = True,drop = False)
itin.info() 




c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\sharrow\accessors.py:25: AccessorRegistrationWarning: registration of accessor <function register_dataarray_method.<locals>.wrapper at 0x000002614937F820> under name 'to_zarr' for type <class 'xarray.core.dataarray.DataArray'> is overriding a preexisting attribute with the same name.
  return xr.register_dataarray_accessor(func.__name__)(wrapper)
c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\numba\model.py:23: UserWarning: 

### larch.numba is experimental, and not feature-complete ###
 the first time you import on a new system, this package will
 compile optimized binaries for your machine, which may take 
 a little while, please be patient 

  warnings.warn( ### EXPERIMENTAL ### )


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6023 entries, (1, 1) to (105, 51)
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   choice        6023 non-null   int64  
 1   traveler      6023 non-null   int64  
 2   origin        6023 non-null   int64  
 3   destination   6023 non-null   int64  
 4   nb_cnxs       6023 non-null   int64  
 5   elapsed_time  6023 non-null   int64  
 6   fare_hy       6023 non-null   float64
 7   fare_ly       6023 non-null   float64
 8   equipment     6023 non-null   int64  
 9   carrier       6023 non-null   int64  
 10  timeperiod    6023 non-null   int64  
dtypes: float64(2), int64(9)
memory usage: 545.3 KB


In [3]:
d = lx.Dataset.construct.from_idce(
    pd.read_csv(lx.example_file("arc"), index_col=['id_case','id_alt']),
)

In [224]:
type(d)

xarray.core.dataset.Dataset

In [4]:

m = lx.Model(datatree=d)

# Define some variable
v = ["timeperiod==2",
    "timeperiod==3",
    "timeperiod==4",
    "timeperiod==5",
    "timeperiod==6",
    "timeperiod==7",
    "timeperiod==8",
    "timeperiod==9",
    "carrier==2",
    "carrier==3",
    "carrier==4",
    "carrier==5",
    "equipment==2",
    "fare_hy",    
    "fare_ly",    
    "elapsed_time",  
    "nb_cnxs",       
]



c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\dataset\flow.py:264: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  case_index = case_ptr_to_indexes(self._obj.dims[self.CASEALT], self[self.CASEPTR].values)


In [5]:
from larch.roles import PX
m.utility_ca = sum(PX(i) for i in v)
m.choice_ca_var = 'choice'
m.availability_var = 1




In [6]:
m.utility_functions()

c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\numba\data_arrays.py:475: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  model_dataset.dims.get(casealt_dim),
c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\dataset\__init__.py:1149: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  return self.root_dataset.dims[self.ALTID]
c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\dataset\__init__.py:1149: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArr

+ P('timeperiod==2') * X('timeperiod==2') + P('timeperiod==3') * X('timeperiod==3') + P('timeperiod==4') * X('timeperiod==4') + P('timeperiod==5') * X('timeperiod==5') + P('timeperiod==6') * X('timeperiod==6') + P('timeperiod==7') * X('timeperiod==7') + P('timeperiod==8') * X('timeperiod==8') + P('timeperiod==9') * X('timeperiod==9') + P('carrier==2') * X('carrier==2') + P('carrier==3') * X('carrier==3') + P('carrier==4') * X('carrier==4') + P('carrier==5') * X('carrier==5') + P('equipment==2') * X('equipment==2') + P.fare_hy * X.fare_hy + P.fare_ly * X.fare_ly + P.elapsed_time * X.elapsed_time + P.nb_cnxs * X.nb_cnxs


In [265]:
itin.columns

Index(['choice', 'traveler', 'origin', 'destination', 'nb_cnxs',
       'elapsed_time', 'fare_hy', 'fare_ly', 'equipment', 'carrier',
       'timeperiod'],
      dtype='object')

In [7]:
m.fit(itin,y = itin.choice)

c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\model\model.py:289: RuntimeWarning: invalid value encountered in cast
  dfs = dfs1.make_dataframes(self.required_data())
converting data_ce to <class 'numpy.float64'>
c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\numba\data_arrays.py:475: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  model_dataset.dims.get(casealt_dim),
c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\dataset\__init__.py:1149: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  return self.root_dataset.dims[self.ALTID]
c:\Users\jiyav

,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
carrier==2,0.117200,0.0,0.0,-inf,inf,0,,0.117200
carrier==3,0.638554,0.0,0.0,-inf,inf,0,,0.638554
carrier==4,0.565252,0.0,0.0,-inf,inf,0,,0.565252
carrier==5,-0.624022,0.0,0.0,-inf,inf,0,,-0.624022
elapsed_time,-0.006087,0.0,0.0,-inf,inf,0,,-0.006087
equipment==2,0.466305,0.0,0.0,-inf,inf,0,,0.466305
fare_hy,-0.001175,0.0,0.0,-inf,inf,0,,-0.001175
fare_ly,-0.001177,0.0,0.0,-inf,inf,0,,-0.001177
nb_cnxs,-2.947153,0.0,0.0,-inf,inf,0,,-2.947153
timeperiod==2,0.095949,0.0,0.0,-inf,inf,0,,0.095949


c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\numba\model.py:1789: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self._check_if_best(result.ll)
c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\numba\model.py:

<larch.NumbaModel (MNL)>

In [244]:
itin

choice  traveler  origin  destination  nb_cnxs  elapsed_time  \
id_case id_alt                                                                 
1       1            0         1     444          222        1           300   
        2            0         1     444          222        1           345   
        3            0         1     444          222        1           335   
        4            0         1     444          222        1           435   
        5            0         1     444          222        1           710   
...                ...       ...     ...          ...      ...           ...   
105     47          30         2     777          111        0           120   
        48          73         2     777          111        0           120   
        49          59         2     777          111        0           120   
        50          32         2     777          111        0           120   
        51          52         2     777          111        0           120   

                  fare_hy    fare_ly  equipment  carrier  timeperiod  
id_case id_alt                                                        
1       1       470.55658    0.00000          1        3           7  
        2       475.92258    0.00000          2        3           5  
        3       443.48166    0.00000          1        3           2  
        4       433.56735    0.00000          2        3           2  
        5       449.83664    0.00000          2        3           2  
...                   ...        ...        ...      ...         ...  
105     47        0.00000  132.52933          2        5           7  
        48        0.00000  169.36240          2        5           8  
        49        0.00000  129.25569          2        5           9  
        50        0.00000  152.32494          2        5           2  
        51        0.00000  159.89514          2        5           7  

[6023 rows x 11 columns]

In [267]:
itin_pred = itin.copy()

In [268]:
d

<xarray.Dataset> Size: 403kB
Dimensions:       (_casealt_: 6023, id_case: 105, id_alt: 127, _caseptr_: 106)
Coordinates:
  * id_case       (id_case) int64 840B 1 2 3 4 5 6 7 ... 100 101 102 103 104 105
  * id_alt        (id_alt) int64 1kB 1 2 3 4 5 6 7 ... 122 123 124 125 126 127
    alt_idx       (_casealt_) int16 12kB 0 1 2 3 4 5 6 ... 44 45 46 47 48 49 50
  * _caseptr_     (_caseptr_) int64 848B 0 67 134 200 ... 5885 5921 5972 6023
Dimensions without coordinates: _casealt_
Data variables:
    choice        (_casealt_) int64 48kB 0 0 0 0 0 0 0 ... 74 61 30 73 59 32 52
    traveler      (id_case) int64 840B 1 2 1 2 1 2 1 2 1 2 ... 2 1 2 2 1 2 2 1 2
    origin        (id_case) int64 840B 444 444 444 444 444 ... 777 777 777 777
    destination   (id_case) int64 840B 222 222 222 222 222 ... 111 111 111 111
    nb_cnxs       (_casealt_) int64 48kB 1 1 1 1 1 1 1 1 1 ... 0 0 0 0 0 0 0 0 0
    elapsed_time  (_casealt_) int64 48kB 300 345 335 435 710 ... 120 120 120 120
    fare_hy       (_casealt_) float64 48kB 470.6 475.9 443.5 ... 0.0 0.0 0.0
    fare_ly       (_casealt_) float64 48kB 0.0 0.0 0.0 0.0 ... 129.3 152.3 159.9
    equipment     (_casealt_) int64 48kB 1 2 1 2 2 1 2 2 2 ... 2 2 2 2 2 2 2 2 2
    carrier       (_casealt_) int64 48kB 3 3 3 3 3 3 3 3 3 ... 5 5 5 5 5 5 5 5 5
    timeperiod    (_casealt_) int64 48kB 7 5 2 2 2 5 6 4 3 ... 4 5 6 6 7 8 9 2 7
Attributes:
    _exclude_dims_:  ('id_case', 'id_alt', '_caseptr_')
    _caseid_:        id_case
    _altid_:         id_alt
    _casealt_:       _casealt_
    _alt_idx_:       alt_idx
    _caseptr_:       _caseptr_

In [269]:
type(itin_pred)

pandas.core.frame.DataFrame

In [270]:
predictions = m.predict(itin)

req_data does not request avail_ca or avail_co but it is set and being provided
converting data_ce to <class 'numpy.float64'>
c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\numba\data_arrays.py:475: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  model_dataset.dims.get(casealt_dim),
c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\dataset\__init__.py:1149: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  return self.root_dataset.dims[self.ALTID]
c:\Users\jiyav\anaconda3\envs\larch_env\lib\site-packages\larch\dataset\flow.py:339: FutureWarning: The return type of 

AttributeError: 'larch.dataframes.DataFrames' object has no attribute '_data_av'

cant use the approach in this because we dont have a fixed number of approaches

PROBLEM:
The outputs in the data summaries for IDCA data currently ignore the alternative availability criteria. Is there a good way we can make these tables exclude the unavailable alternatives?